In [29]:
import pandas as pd
import geopandas
import numpy as np

from tqdm import tqdm
from shapely.geometry import Point
from geopy.distance import great_circle
from scipy import stats
from multiprocessing import Pool

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('seaborn-muted')

In [4]:
df = pd.read_table('../data/Gaz_places_national.txt', encoding='ISO-8859-1')

In [5]:
df.head(10)

,USPS,GEOID,ANSICODE,NAME,LSAD,FUNCSTAT,POP10,HU10,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,100100,2582661,Abanda CDP,57,S,192,79,7764035,34284,2.998,0.013,33.091627,-85.527029
1,AL,100124,2403054,Abbeville city,25,A,2688,1255,40255352,107642,15.543,0.042,31.564216,-85.259634
2,AL,100460,2403063,Adamsville city,25,A,4522,1990,65083153,29719,25.129,0.011,33.605750,-86.974650
3,AL,100484,2405123,Addison town,43,A,758,351,9753286,83426,3.766,0.032,34.202689,-87.177901
4,AL,100676,2405125,Akron town,43,A,356,205,1778126,13850,0.687,0.005,32.879417,-87.741757
5,AL,100820,2403069,Alabaster city,25,A,30352,11295,64860720,747585,25.043,0.289,33.214355,-86.823080
6,AL,100988,2403074,Albertville city,25,A,21160,8128,68780663,258708,26.556,0.100,34.263130,-86.210660
7,AL,101132,2403077,Alexander City city,25,A,14875,6834,105771865,737649,40.839,0.285,32.927240,-85.937122
8,AL,101180,2402638,Alexandria CDP,57,S,3917,1599,28786548,33123,11.115,0.013,33.765175,-85.879596
9,AL,101228,2403080,Aliceville city,25,A,2486,1164,11814094,0,4.561,0.000,33.123744,-88.159445


In [25]:
class DensityScorer:
    
    def __init__(self, path):
        
        df = pd.read_table(path, encoding='ISO-8859-1')
        
        self.cities = [
            (r.INTPTLAT, r.INTPTLONG, r.POP10)
            for r in df.itertuples()
        ]
        
    def __repr__(self):
        return '%s<%d cities>' % (self.__class__.__name__, len(self.cities))
    
    def __call__(self, lat, lon, std=20):
        
        score = 0
        for tlat, tlon, pop in tqdm(self.cities):
            d = great_circle((lat, lon), (tlat, tlon)).miles
            w = stats.norm.pdf(d, 0, std)
            score += pop * w
            
        return score

In [26]:
ds = DensityScorer('../data/Gaz_places_national.txt')

In [27]:
ds

DensityScorer<29514 cities>

In [28]:
ds(34.04, -118.27)

100%|██████████| 29514/29514 [00:03<00:00, 7722.40it/s]


176320.22477088476

In [19]:
ds(36.07, -119.11)

100%|██████████| 29514/29514 [00:03<00:00, 7740.12it/s]


6944.444688714793

In [20]:
ds(33.39, -87.53)

100%|██████████| 29514/29514 [00:03<00:00, 7788.78it/s]


4059.0853360338015

In [24]:
ds(38.80, -80.80)

100%|██████████| 29514/29514 [00:03<00:00, 7684.16it/s]


504.96083798822457

In [32]:
ds(36.73, -119.73)

100%|██████████| 29514/29514 [00:03<00:00, 7792.20it/s]


16859.59397502989

In [33]:
ds(38.55, -121.56)

100%|██████████| 29514/29514 [00:03<00:00, 7803.05it/s]


34587.89816186251

In [36]:
r = ds(42.52, -115.99)

100%|██████████| 29514/29514 [00:06<00:00, 4222.28it/s]


In [39]:
type(float(r))

float